In [1]:
pip install facenet-pytorch


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.9/1.9 MB 5.4 MB/s eta 0:00:00m eta 0:00:010:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.9/6.9 MB 14.8 MB/s eta 0:00:0031m14.9 MB/s eta 0:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 755.5/755.5 MB 2.3 MB/s eta 0:00:00m eta 0:00:010:00:05
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 731.7/731.7 MB 5.4 MB/s eta 0:00:00m eta 0:00:010:00:02
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 166.0/166.0 MB 12.2 MB/s eta 0:00:00m eta 0:00:010:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 167.9/167.9 MB 10.9 MB/s eta 0:00:00m eta 0:00:010:00:01
  Attempting uninstall: triton
    Found existing installation: triton 3.1.0
    Uninstalling triton-3.1.0:
      Successfully uninstalled triton-3.1.0
  Attempting uninstall: nvidia-nccl-cu12
    Found existing installation: nvidia-nccl-cu12 2.21.5
    Uninstalling nvidia-nccl-cu12-2.21.5:
      Successfully uninstalled nvidia-nccl-cu12-2.21.5
  Attempting uninstall: nvidia-cudnn-cu12
    Fo

In [13]:
import os
from facenet_pytorch import MTCNN
from PIL import Image

# Initialize MTCNN for face detection
mtcnn = MTCNN(keep_all=True)

def process_and_save_faces(image_path, output_base_dir):
    """Detect and save all faces without filtering."""
    img = Image.open(image_path).convert("RGB")
    faces, _ = mtcnn.detect(img)

    if faces is None:
        print(f"❌ No faces detected in {image_path}")
        return

    # Preserve folder structure
    relative_path = os.path.relpath(image_path, input_base_dir)
    output_dir = os.path.join(output_base_dir, os.path.dirname(relative_path))
    os.makedirs(output_dir, exist_ok=True)

    # Save each detected face
    filename, ext = os.path.splitext(os.path.basename(image_path))
    for i, box in enumerate(faces):
        face = img.crop(box)
        output_file = os.path.join(output_dir, f"{filename}_face{i+1}{ext}")
        face.save(output_file)
        print(f"✅ Saved: {output_file}")

# Define input and output directories
input_base_dir = "/mnt/documents/Minor-Project/Datasets/FF++Snaps"
output_base_dir = "/mnt/documents/Minor-Project/Datasets/TEST"

# Walk through all folders (train, test, validation → fake/real)
for root, _, files in os.walk(input_base_dir):
    for file in files:
        if file.lower().endswith(('.png', '.jpg', '.jpeg')):
            image_path = os.path.join(root, file)
            process_and_save_faces(image_path, output_base_dir)


✅ Saved: /mnt/documents/Minor-Project/Datasets/TEST/validation/fake/video_0176_frame_0013_face1.png
✅ Saved: /mnt/documents/Minor-Project/Datasets/TEST/validation/fake/video_0189_frame_0027_face1.png
✅ Saved: /mnt/documents/Minor-Project/Datasets/TEST/validation/fake/video_0171_frame_0008_face1.png
✅ Saved: /mnt/documents/Minor-Project/Datasets/TEST/validation/fake/video_0186_frame_0019_face1.png
✅ Saved: /mnt/documents/Minor-Project/Datasets/TEST/validation/fake/video_0178_frame_0000_face1.png
✅ Saved: /mnt/documents/Minor-Project/Datasets/TEST/validation/fake/video_0178_frame_0000_face2.png
✅ Saved: /mnt/documents/Minor-Project/Datasets/TEST/validation/fake/video_0191_frame_0003_face1.png
✅ Saved: /mnt/documents/Minor-Project/Datasets/TEST/validation/fake/video_0182_frame_0007_face1.png
✅ Saved: /mnt/documents/Minor-Project/Datasets/TEST/validation/fake/video_0195_frame_0011_face1.png
✅ Saved: /mnt/documents/Minor-Project/Datasets/TEST/validation/fake/video_0190_frame_0010_face1.png


In [1]:
import os
from facenet_pytorch import MTCNN
from PIL import Image
import torch

# Initialize MTCNN to return bounding boxes
mtcnn = MTCNN(keep_all=False, select_largest=True)

def process_and_save_faces(image_path, output_base_dir):
    """Detect and save a single face from each image."""
    try:
        # Load image
        img = Image.open(image_path).convert("RGB")
        
        # Detect face and get bounding boxes
        boxes, _ = mtcnn.detect(img)
        
        if boxes is None:
            print(f"❌ No face detected in {image_path}")
            return
            
        # Get coordinates for the largest face
        box = boxes[0]
        x1, y1, x2, y2 = [int(coord) for coord in box]
        
        # Crop the face from original image
        face = img.crop((x1, y1, x2, y2))
        
        # Preserve folder structure
        relative_path = os.path.relpath(image_path, input_base_dir)
        output_dir = os.path.join(output_base_dir, os.path.dirname(relative_path))
        os.makedirs(output_dir, exist_ok=True)
        
        # Save the face using the original filename
        filename, ext = os.path.splitext(os.path.basename(image_path))
        output_file = os.path.join(output_dir, f"{filename}{ext}")
        face.save(output_file)
        print(f"✅ Saved: {output_file}")
        
    except Exception as e:
        print(f"❌ Error processing {image_path}: {str(e)}")

# Define input and output directories
input_base_dir = "/mnt/documents/Minor-Project/Datasets/FF++Snaps"
output_base_dir = "/mnt/documents/Minor-Project/Datasets/FF++Faces"

# Walk through all folders
for root, _, files in os.walk(input_base_dir):
    for file in files:
        if file.lower().endswith(('.png', '.jpg', '.jpeg')):
            image_path = os.path.join(root, file)
            process_and_save_faces(image_path, output_base_dir)

/mnt/documents/Minor-Project/Codes/myenv/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


✅ Saved: /mnt/documents/Minor-Project/Datasets/FF++Faces/validation/fake/video_0176_frame_0013.png
✅ Saved: /mnt/documents/Minor-Project/Datasets/FF++Faces/validation/fake/video_0189_frame_0027.png
✅ Saved: /mnt/documents/Minor-Project/Datasets/FF++Faces/validation/fake/video_0171_frame_0008.png
✅ Saved: /mnt/documents/Minor-Project/Datasets/FF++Faces/validation/fake/video_0186_frame_0019.png
✅ Saved: /mnt/documents/Minor-Project/Datasets/FF++Faces/validation/fake/video_0178_frame_0000.png
✅ Saved: /mnt/documents/Minor-Project/Datasets/FF++Faces/validation/fake/video_0191_frame_0003.png
✅ Saved: /mnt/documents/Minor-Project/Datasets/FF++Faces/validation/fake/video_0182_frame_0007.png
✅ Saved: /mnt/documents/Minor-Project/Datasets/FF++Faces/validation/fake/video_0195_frame_0011.png
✅ Saved: /mnt/documents/Minor-Project/Datasets/FF++Faces/validation/fake/video_0190_frame_0010.png
✅ Saved: /mnt/documents/Minor-Project/Datasets/FF++Faces/validation/fake/video_0182_frame_0006.png
✅ Saved: /